In [1]:
# Disclaimer: This project uses synthetic data for demonstration purposes only.
# This notebook prepares the data for model training, building upon the feature engineering
# performed in BigQuery.

import pandas as pd
import pandas_gbq
from google.colab import auth
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# --- Connect to BigQuery and load the prepared data view ---
print("Authenticating and loading prepared data from BigQuery...")
auth.authenticate_user()
project_id = "driiiportfolio"
query = f"SELECT * FROM `{project_id}.sds_project.project_ready_data`"
df = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

print("Data loaded successfully. Shape:", df.shape)
print("\nFirst 5 rows of the prepared data:")
print(df.head())

# --- Handle Categorical Features ---
# Identify categorical columns to encode
categorical_features = ['product_sku', 'shipping_country', 'manufacturing_plant']

# Use OneHotEncoder to transform categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = encoder.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Drop original categorical columns and concatenate with encoded features
df_final = df.drop(columns=categorical_features).reset_index(drop=True)
df_final = pd.concat([df_final, encoded_df], axis=1)

print("\nFinal DataFrame after one-hot encoding:")
print(df_final.head())
print("\nFinal DataFrame shape:", df_final.shape)

# --- Define Features (X) and Target (y) ---
X = df_final.drop(columns=['order_id', 'customer_id', 'is_delayed'])
y = df_final['is_delayed']

# --- Check for Class Imbalance ---
class_counts = y.value_counts(normalize=True)
print("\nClass distribution of the target variable 'is_delayed':")
print(class_counts)

# Note to stakeholders: The data shows a class imbalance.
# The `modeling_analysis` phase will address this by using an appropriate
# model and evaluation metrics.

# --- Split Data into Training and Testing Sets ---
# We use 'stratify=y' to ensure the train and test sets have the same proportion of delayed orders.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nData splitting complete.")
print(f"Training set size: {X_train.shape[0]} rows")
print(f"Testing set size: {X_test.shape[0]} rows")

# The X_train, X_test, y_train, and y_test DataFrames are now ready for model training.

Authenticating and loading prepared data from BigQuery...
Downloading: 100%|██████████|
Data loaded successfully. Shape: (50000, 14)

First 5 rows of the prepared data:
     order_id customer_id  is_delayed        product_sku  ram_config_gb  \
0  ORD-000111  CUST-32662           0  Mac Studio M3 Max             16   
1  ORD-000239  CUST-11802           0  Mac Studio M3 Max             64   
2  ORD-001164  CUST-30806           0  Mac Studio M3 Max             64   
3  ORD-004357  CUST-73610           1  Mac Studio M3 Max             16   
4  ORD-007336  CUST-38690           0  Mac Studio M3 Max             32   

   storage_config_gb  is_custom_config shipping_country manufacturing_plant  \
0               1024              True              USA       China Plant A   
1               1024              True          Germany       China Plant A   
2                512              True            Japan       India Plant B   
3               2048              True            China     Viet